In [16]:
import os
import random
import re
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import string
import collections
from nltk.stem.porter import *

In [17]:
# Randomly select 10 10-K from those downloaded last week
f_list = []
path = 'C:/Users/jlxle/Desktop/files'   
for i in os.listdir(path):  
    f_list.append(i)
fr_list = random.sample(f_list,10)    
print(fr_list)

['10-K--1086222--2020-02-28--0001086222-20-000045.html', '10-K--880984--2020-03-25--0001493152-20-004708.html', '10-K--1337553--2020-02-24--0001628280-20-002027.html', '10-K--899629--2020-02-21--0001564590-20-005641.html', '10-K--1785173--2020-03-18--0001564590-20-011636.html', '10-K--1004434--2020-02-28--0001004434-20-000011.html', '10-K--910638--2020-02-26--0000910638-20-000010.html', '10-K--1622996--2020-01-07--0001640334-20-000007.html', '10-K--1769624--2020-03-31--0001213900-20-008207.html', '10-K--1326321--2020-03-30--0000931755-20-000013.html']


In [18]:
# Create two dataframe, which will be written into 2 .csv files
df_w = pd.read_csv('C:/Users/jlxle/Desktop/week2/totalwordcountd.csv')
df_n = pd.read_excel('C:/Users/jlxle/Desktop/week2/LoughranMcDonald_SentimentWordLists_2018.xlsx',sheet_name='Negative',header=None,names=['NegativeWords','CountAllDocument'])

In [19]:
print(df_w[:3])
print(df_n[:3])

  FileName  WordCount  TotalNegativeWords  TotalStemmedNegativeWords  \
0   1.html        NaN                 NaN                        NaN   
1   2.html        NaN                 NaN                        NaN   
2   3.html        NaN                 NaN                        NaN   

   NegativeWordRatio  StemmedNegativeWordRatio  
0                NaN                       NaN  
1                NaN                       NaN  
2                NaN                       NaN  
  NegativeWords  CountAllDocument
0       ABANDON               NaN
1     ABANDONED               NaN
2    ABANDONING               NaN


In [13]:
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [35]:
WordCount = []
occur = []  
TotalNegativeWords = []
TotalStemmedNegativeWords = []

for i in fr_list:
    print(i)
    
    # Clean textual data using Beautiful Soup
    f = open(path + '/' + i,"r",encoding='UTF8').read()    
    sp = BeautifulSoup(f,'html.parser')    
    spt = sp.get_text()              # After applying .get_text(), at the end of each file, beginning with 'EX-101',still messy
    start_loc = spt.find('EX-101')   # delete all the messy and useless data after 'EX-101', leaving items and  
    spt = spt[:start_loc]            # some exhibits, which are in good format
    spt = re.sub(r'[\!\"\#\$\%\&\'\(\)\*\+\,\.\/\:\;\<\=\>\?\@\\\]\[\^\_\`\{\|\}\~\-\–\—\d\§]', '', spt)     
                                        # Remove punctuation and numbers                    
    spt_list = spt.upper().split()      # Convert words to uppercase and split word token
    WordCount.append(len(spt_list))     # count words for each file and append the count to a list
    
    countNeg = 0
    for j in spt_list: 
        if j in df_n['NegativeWords'].values:   
            occur.append(j)
            countNeg += 1
    TotalNegativeWords.append(countNeg)   
    
    countStem = 0       
    stemmer = PorterStemmer()          # Stemming
    spt_list_stem = [stemmer.stem(i) for i in spt_list]
    n_stem = [stemmer.stem(i) for i in df_n['NegativeWords'].values]
    for j in spt_list_stem:
        if j in n_stem:
            countStem += 1  
    TotalStemmedNegativeWords.append(countStem)        

10-K--1086222--2020-02-28--0001086222-20-000045.html
10-K--880984--2020-03-25--0001493152-20-004708.html
10-K--1337553--2020-02-24--0001628280-20-002027.html
10-K--899629--2020-02-21--0001564590-20-005641.html
10-K--1785173--2020-03-18--0001564590-20-011636.html
10-K--1004434--2020-02-28--0001004434-20-000011.html
10-K--910638--2020-02-26--0000910638-20-000010.html
10-K--1622996--2020-01-07--0001640334-20-000007.html
10-K--1769624--2020-03-31--0001213900-20-008207.html
10-K--1326321--2020-03-30--0000931755-20-000013.html


In [36]:
cnt = collections.Counter(occur)   # dictionary，key--negative words，value--occurence in all 10 files     
k = df_n['NegativeWords'].values
n = 0
for j in k:
    df_n.loc[n,'CountAllDocument'] = cnt[j]       # How many times each negative words appear in all 10 files
    n += 1

In [37]:
# Put values into dataframe
df_w['FileName'] = fr_list        
df_w['WordCount'] = WordCount
df_w['TotalNegativeWords'] = TotalNegativeWords
df_w['TotalStemmedNegativeWords'] = TotalStemmedNegativeWords

In [38]:
# axis =1 ,apply function to each row; axis =0(default), column
df_w['NegativeWordRatio'] = df_w[['TotalNegativeWords','WordCount']].apply(lambda x : x['TotalNegativeWords'] / x['WordCount'] ,axis=1 ) 
df_w['StemmedNegativeWordRatio'] = df_w[['TotalStemmedNegativeWords','WordCount']].apply(lambda x : x['TotalStemmedNegativeWords'] / x['WordCount'] ,axis=1 ) 

In [39]:
print(df_w[:3])

                                            FileName  WordCount  \
0  10-K--1086222--2020-02-28--0001086222-20-00004...      60091   
1  10-K--880984--2020-03-25--0001493152-20-004708...      36550   
2  10-K--1337553--2020-02-24--0001628280-20-00202...      97499   

   TotalNegativeWords  TotalStemmedNegativeWords  NegativeWordRatio  \
0                 845                       1316           0.014062   
1                 550                        803           0.015048   
2                1978                       2980           0.020287   

   StemmedNegativeWordRatio  
0                  0.021900  
1                  0.021970  
2                  0.030564  


In [40]:
df_n.to_csv('C:/Users/jlxle/Desktop/NegativeWordsCount.csv', sep=",", index=False )
df_w.to_csv('C:/Users/jlxle/Desktop/TotalWordsCount.csv', sep="," , index=False )